## import & 데이터 로드

In [5]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드
df = pd.read_csv('ML_olist.csv')
print(f"데이터 크기: {df.shape}")
print(f"\n컬럼 목록: {df.columns.tolist()}")
df.head()


데이터 크기: (64850, 13)

컬럼 목록: ['order_id', 'order_approved_at', 'order_delivered_carrier_date', 'seller_id', 'shipping_limit_date', 'product_category_name_english', 'review_score', 'review_comment_message', 'has_text_review', 'seller_processing_days', 'is_logistics_fault', 'seller_delay_days', 'processing_days_diff']


,order_id,order_approved_at,order_delivered_carrier_date,seller_id,shipping_limit_date,product_category_name_english,review_score,review_comment_message,has_text_review,seller_processing_days,is_logistics_fault,seller_delay_days,processing_days_diff
0,53cdb2fc8bc7dce0b6741e2150273451,2018-07-26 03:24:27,2018-07-26 14:31:00,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,perfumery,4.0,Muito bom o produto.,True,0.46,False,-3.54,-1.85
1,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:55:23,2018-08-08 13:50:00,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,auto,5.0,NaN,False,0.20,False,-4.80,-2.09
2,a4591c265e18cb1dcee52889e2d8acc3,2017-07-09 22:10:13,2017-07-11 14:58:04,8581055ce74af1daba164fdbd55a40de,2017-07-13 22:10:13,auto,4.0,NaN,False,1.70,False,-2.30,-0.59
3,6514b8ad8028c9f2cc2374ded245783f,2017-05-16 13:22:11,2017-05-22 10:07:46,16090f2ca825584b5a147ab24aa30c86,2017-05-22 13:22:11,auto,5.0,NaN,False,5.86,False,-0.14,3.57
4,76c6e866289321a7c93b82b54852dc33,2017-01-25 02:50:47,2017-01-26 14:16:31,63b9ae557efed31d1f7687917d248a8d,2017-01-27 18:29:09,furniture_decor,1.0,NaN,False,1.48,False,-1.18,-1.57


## XGBClassifier (Extreme Gradient Boosting Classification) 
- 트리 기반 앙상블 모델 
- > 몇 일부터 위험한가?

- target : 리뷰 스코어 1-3점이면 1(불만족), 4~5점이면 0(만족) 

- features 
- 직접적 지연 지표 : seller_delay_days, processing_days_diff 
- 운영 지표 : seller_processing_days, is_logistics_fault 
- 정황 지표 : has_test_review (과거 패턴 분석용) 

In [13]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# 데이터 로드 + 타겟 생성
df = pd.read_csv('ML_olist.csv')
df['is_at_risk'] = df['review_score'].apply(lambda x: 1 if x <= 3 else 0)

# 피처 선택 (지연 없음 원칙 기반 변수 중심)
features = ['seller_delay_days', 'processing_days_diff', 'seller_processing_days', 'is_logistics_fault']
X = df[features]
y = df['is_at_risk']

# 학습/테스트 셋 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 모델 (XGBoost)
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)

# 성능 평가
preds = model.predict(X_test)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89     10378
           1       0.65      0.06      0.11      2592

    accuracy                           0.81     12970
   macro avg       0.73      0.53      0.50     12970
weighted avg       0.78      0.81      0.73     12970



In [14]:
# 과거에 지연 없음을 어겼을 때, 불만족 리뷰가 달렸던 패턴을 학습해, 앞으로 문제가 생길 판매자를 미리 짚어냄 

# 0 : 정상 판매자 
# 1 : 유의 판매자 (1~3점 받은 판매자) 

# precision = 실제 불만족 리뷰가 발생한 비율 (0.65)
# recall = 실제 불만족 리뷰 만든 판매자 중 모델이 잡아낸 비율 (0.06) 

# 유의 판매자라고 확신하는 경우의 정확도는 괜찮으나, 유의 판매자 잘 못잡아냄 
# 재현율 0.11 -> 망함. ㅠ 

## is_logistics_fault = False 조건(판매자 과실에 집중)학습

In [18]:
# 물류사 과실이 없는 데이터만 필터링 (순수 판매자 운영 데이터)
# 'is_logistics_fault'가 False인 행만 추출
df_seller = df[df['is_logistics_fault'] == False].copy()

# 타겟(Y) 및 피처(X) 설정
# 불만족 리뷰(1~3점)를 1로, 만족 리뷰(4~5점)를 0으로 라벨링
df_seller['is_at_risk'] = df_seller['review_score'].apply(lambda x: 1 if x <= 3 else 0)

# 분석에 사용할 핵심 파생변수 선택
features = ['seller_delay_days', 'processing_days_diff', 'seller_processing_days']
X = df_seller[features]
y = df_seller['is_at_risk']

# 학습/테스트 데이터 분리 (불균형 데이터이므로 stratify 적용)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 모델 학습  (XGBoost)
# scale_pos_weight는 데이터 불균형을 해결하기 위해 클래스 비중을 조절하는 옵션입니다.
model = XGBClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=5, 
    scale_pos_weight=(len(y) - sum(y)) / sum(y), # 0과 1의 비율을 맞춰줌
    random_state=42
)
model.fit(X_train, y_train)

# 결과 출력
y_pred = model.predict(X_test)

print("--- [판매자 과실 집중 모델] 성능 성적표 ---")
print(classification_report(y_test, y_pred))

# 변수 중요도 확인 (어떤 지표가 유의 판매자를 가장 잘 잡아내는가?)
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
print("\n--- 변수 중요도  ---")
print(importances)

--- [판매자 과실 집중 모델] 성능 성적표 ---
              precision    recall  f1-score   support

           0       0.83      0.64      0.72     10137
           1       0.22      0.45      0.30      2341

    accuracy                           0.60     12478
   macro avg       0.53      0.55      0.51     12478
weighted avg       0.72      0.60      0.64     12478


--- 변수 중요도  ---
seller_processing_days    0.441297
processing_days_diff      0.287455
seller_delay_days         0.271248
dtype: float32


In [ ]:
# 변수 중요도가 0.5이하로 낮게 나오는 이유
# seller_delay_days(절대적)와 processing_days_diff(상대적)이 정보를 나눠 가지고 있기 때문 
# processing_days_diff가 +5가 되는 등 특정 구간에서만 위력을 발휘할 가능성이 큼 

## 텍스트 유무별 비교 학습

In [19]:
df_seller = df[df['is_logistics_fault'] == False].copy()
df_seller['is_at_risk'] = df_seller['review_score'].apply(lambda x: 1 if x <= 3 else 0)

features = ['seller_delay_days', 'processing_days_diff', 'seller_processing_days']

# 데이터를 텍스트 유무로 분리
for has_text in [True, False]:
    subset = df_seller[df_seller['has_text_review'] == has_text]
    
    X = subset[features]
    y = subset['is_at_risk']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    model = XGBClassifier(
        n_estimators=100, 
        scale_pos_weight=(len(y) - sum(y)) / sum(y),
        random_state=42
    )
    model.fit(X_train, y_train)
    
    print(f"\n=== [has_text_review: {has_text}] 모델 결과 ===")
    print(classification_report(y_test, model.predict(X_test)))


=== [has_text_review: True] 모델 결과 ===
              precision    recall  f1-score   support

           0       0.77      0.66      0.71      3756
           1       0.39      0.52      0.45      1556

    accuracy                           0.62      5312
   macro avg       0.58      0.59      0.58      5312
weighted avg       0.66      0.62      0.63      5312


=== [has_text_review: False] 모델 결과 ===
              precision    recall  f1-score   support

           0       0.91      0.71      0.80      6381
           1       0.15      0.41      0.22       785

    accuracy                           0.68      7166
   macro avg       0.53      0.56      0.51      7166
weighted avg       0.82      0.68      0.73      7166



In [20]:
# 1 = 유의 판매자 

# has_text_true = 예측도 0.39 / 잡는정도 0.52 
# has_text_false = 예측정도 0.15 / 잡는정도 0.4 (탈락)

## smote 적용 (오버샘플링)

- 기준1 : 판매자 과실만 측정 (is_logistics_fault = False)
- 기준2 : 텍스트 리뷰 유무 분리 (has_text_review)
- 기준3 : smote (5:5)

- 사용한 파생변수 = seller_delay_days(절대 지연), processing_days_diff(상대적 속도), seller_processing_days(순수 처리일)

In [24]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# SMOTE 생성
# 유의 판매자(1) 데이터를 정상 데이터(0)와 동일한 양이 될 때까지 생성
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

print(f"샘플링 전 클래스 비율: {Counter(y_train)}")
print(f"샘플링 후 클래스 비율: {Counter(y_resampled)}")

# 샘플링된 데이터로 모델 재학습
model_smote = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
model_smote.fit(X_resampled, y_resampled)

# 결과 확인
y_pred_smote = model_smote.predict(X_test)
print("\n--- SMOTE 적용 - 모델 성능 성적표 ---")
print(classification_report(y_test, y_pred_smote))

샘플링 전 클래스 비율: Counter({0: 25526, 1: 3138})
샘플링 후 클래스 비율: Counter({1: 25526, 0: 25526})

--- SMOTE 적용 - 모델 성능 성적표 ---
              precision    recall  f1-score   support

           0       0.90      0.65      0.76      6381
           1       0.13      0.43      0.20       785

    accuracy                           0.63      7166
   macro avg       0.52      0.54      0.48      7166
weighted avg       0.82      0.63      0.70      7166



## 모델이 사용한 지열 일수의 컷-오프 지점 확인 

In [32]:
# 모델의 의사결정 나무 구조 - 텍스트 추출 
trees = model_smote.get_booster().get_dump()

# 가장 영향력이 큰 기준의 첫 줄 확인
first_tree = trees[0].split('\n')[0]

print("--- 모델 첫 판단 기준 ---")
print(first_tree)

# 변수 이름과 매칭
# f0, f1 등으로 나올 경우를 대비해 피처 순서를 확인
feature_map = {f'f{i}': col for i, col in enumerate(X_train.columns)}
print(f"\n--- 변수 매핑 가이드 ---")
for f_code, col_name in feature_map.items():
    print(f"{f_code} : {col_name}")

--- 모델 첫 판단 기준 ---
0:[seller_processing_days<2.70080185] yes=1,no=2,missing=2

--- 변수 매핑 가이드 ---
f0 : seller_delay_days
f1 : processing_days_diff
f2 : seller_processing_days


In [33]:
# 주문이 들어온 순간부터 판매자가 넘길때까지 걸린 시간이 2.7일이면, 정상 그룹 
# 2.7일로 한 이유 = 이 지점을 기준으로 긍정과 불만족 리뷰 비율이 극적으로 갈라졌기 때문임 

## 카테고리별 임계점 찾기 + 종합 모델링 

In [50]:
def get_category_patience_analysis(df, top_n=40):
    results = []
    top_cats = df['product_category_name_english'].value_counts().head(top_n).index
    
    for cat in top_cats:
        sub = df[df['product_category_name_english'] == cat].copy()
        
        # 0.5일 단위 구간 생성
        sub['days_bin'] = np.floor(sub['seller_processing_days'] * 2) / 2
        status_by_days = sub.groupby('days_bin')['is_at_risk'].agg(['mean', 'count'])
        
        # 임계점: 불만족 비율 0.4 이상, 데이터 5건 이상
        cutoff_points = status_by_days[(status_by_days['mean'] >= 0.3) & (status_by_days['count'] >= 3)]
        cutoff = cutoff_points.index.min() if not cutoff_points.empty else np.nan
        
        results.append({
            'Category': cat,
            'Total_Orders': len(sub),
            'Avg_Processing': round(sub['seller_processing_days'].mean(), 2),
            'Patience_Limit': cutoff
        })
    
    # Patience_Limit 기준으로 오름차순 정렬 (민감한 카테고리가 위로)
    return pd.DataFrame(results).sort_values(by='Patience_Limit', na_position='last')

# 분석 실행
final_analysis_df = get_category_patience_analysis(df_seller, top_n=40)


pd.set_option('display.max_rows', 50)      # 40개 다 보이게 설정
pd.set_option('display.max_columns', None) # 모든 컬럼 표시
pd.set_option('display.width', 1000)       # 줄바꿈 방지

# 인덱스 무시하고 깔끔하게 출력
print(final_analysis_df.to_string(index=False))

                               Category  Total_Orders  Avg_Processing  Patience_Limit
         industry_commerce_and_business           147            2.25             0.5
                  furniture_living_room           297            3.23             0.5
                           home_confort           225            2.25             0.5
                        fixed_telephony           155            2.94             0.5
              construction_tools_lights           196            2.40             1.5
                                 drinks           199            2.59             1.5
                       air_conditioning           178            3.10             2.5
                         consoles_games           502            3.11             3.5
        construction_tools_construction           576            1.76             3.5
kitchen_dining_laundry_garden_furniture           149            2.53             3.5
                       office_furniture           957 

In [53]:
# 불만족 비율 30% 넘어갈 때, 샘플이 3개 이상일때 기준점 산출 

# 불만족 비율 30 - 특정 카테고리에서 배송이 5일 걸린 주문 100건을 모아 1~3점 주문이 몇건인지 세고, 
# 30건을 넘어가는 순간 5일 = 10명중 3명이 화를 낸다 -> patience_limit 

## ai 예측확률과 카테고리별 실제 고객 인내심을 결합해 검증 

In [69]:
def strategic_risk_classifier_fixed(X_data, original_df, xgb_model, category_limits):
    """
    KeyError를 해결하고 원본 카테고리 명칭을 매칭하는 판별 모델
    """
    # XGBoost 예측 확률 계산
    risk_probs = xgb_model.predict_proba(X_data)[:, 1]
    
    #  X_test와 원본 데이터의 인덱스를 기준으로 카테고리명 복원
    analysis_data = X_data.copy()
    analysis_data['xgb_prob'] = risk_probs
    
    # 원본 데이터에서 카테고리와 셀러 ID 합치기
    cat_mapping = original_df[['product_category_name_english', 'seller_id']]
    analysis_data = analysis_data.join(cat_mapping, how='left', rsuffix='_origin')
    
    # 카테고리 마지노선(30% 기준) 결합
    analysis_data = pd.merge(analysis_data, category_limits[['Category', 'Patience_Limit']], 
                             left_on='product_category_name_english', right_on='Category', how='left')
    
    # 마지노선이 없는 경우 전체 평균(2.7일) 적용
    analysis_data['Patience_Limit'] = analysis_data['Patience_Limit'].fillna(2.7)
    
    # 최종 유의 판매자 판정
    # AI 확률이 높고(0.5↑) + 실제 지연이 카테고리별 마지노선을 넘긴(Patience_Limit↑) 경우만 1로 판정
    analysis_data['final_risk_label'] = np.where(
        (analysis_data['xgb_prob'] >= 0.5) & 
        (analysis_data['seller_processing_days'] > analysis_data['Patience_Limit']), 
        1, 0
    )
    
    # 리스크 점수 (0~100점)
    analysis_data['risk_score'] = (analysis_data['xgb_prob'] * 60 + 
                                   (analysis_data['seller_processing_days'] / analysis_data['Patience_Limit']) * 40)
    
    return analysis_data

# 모델 실행
final_model_output = strategic_risk_classifier_fixed(X_test, df_seller, model_smote, final_analysis_df)
print(final_model_output[['seller_id', 'product_category_name_english', 'risk_score', 'final_risk_label']].head(20))

                           seller_id    product_category_name_english  risk_score  final_risk_label
0   0509040ea3fe50071181bbc359eb7738                   sports_leisure   32.248602                 0
1   79ebd9a61bac3eaf882805ed4ecfa12a                  furniture_decor   40.525915                 0
2   4c2b230173bb36f9b240f2b8ac11786e                   sports_leisure   76.124449                 0
3   6cd68b3ed6d59aaa9fece558ad360c0a              luggage_accessories   49.604553                 0
4   c003204e1ab016dfa150abc119207b24                             auto   36.572377                 0
5   282f23a9769b2690c5dda22e316f9941                         pet_shop   43.143278                 0
6   ceaec5548eefc6e23e6607c5435102e7                       cool_stuff   48.484883                 0
7   8f2ce03f928b567e3d56181ae20ae952                   sports_leisure   42.865374                 0
8   004c9cd9d87a3c30c522c48c4fc07416                          unknown   31.730596                 0


In [66]:
def final_strategic_classifier(X_data, original_df, xgb_model, category_limits):
    """
    기획팀 전용: AI 예측과 카테고리별 마지노선을 결합한 고정밀 식별 모델
    """
    # AI 기반 위험 확률 추출 
    risk_probs = xgb_model.predict_proba(X_data)[:, 1]
    
    # 데이터 통합 (카테고리 및 마지노선 매칭)
    analysis_df = X_data.copy()
    analysis_df['xgb_prob'] = risk_probs
    
    # 원본 카테고리 명칭 복원
    cat_info = original_df[['product_category_name_english', 'seller_id']]
    analysis_df = analysis_df.join(cat_info, how='left', rsuffix='_origin')
    
    # 카테고리별 마지노선(30% 기준) 결합
    analysis_df = pd.merge(analysis_df, category_limits[['Category', 'Patience_Limit']], 
                             left_on='product_category_name_english', right_on='Category', how='left')
    
    # 마지노선 데이터 부족 시 플랫폼 기본값(2.7일) 적용
    analysis_df['Patience_Limit'] = analysis_df['Patience_Limit'].fillna(2.7)

    # 이중 필터링 식별 로직
    # 조건 1: AI가 판단한 위험 확률이 50% 이상인가?
    # 조건 2: 실제 지연 시간이 해당 카테고리의 고객 마지노선을 넘겼는가?
    analysis_df['is_confirmed_risk'] = np.where(
        (analysis_df['xgb_prob'] >= 0.5) & 
        (analysis_df['seller_processing_days'] > analysis_df['Patience_Limit']), 
        1, 0
    )
    
    # 관리 우선순위를 위한 리스크 스코어링 (0~100)
    analysis_df['final_priority_score'] = (
        (analysis_df['xgb_prob'] * 0.7) + 
        (np.clip(analysis_df['seller_processing_days'] / analysis_df['Patience_Limit'], 0, 2) * 0.3)
    ) * 100

    return analysis_df[['seller_id', 'product_category_name_english', 'seller_processing_days', 
                        'Patience_Limit', 'final_priority_score', 'is_confirmed_risk']]

# 모델 실행
identified_sellers = final_strategic_classifier(X_test, df_seller, model_smote, final_analysis_df)


# 만약 '유의 판매자'로 확정된 사람(1)만 보기
confirmed_only = identified_sellers[identified_sellers['is_confirmed_risk'] == 1]
print(f"전체 유의 식별 판매자 수: {len(confirmed_only)}명")
print(confirmed_only.sort_values(by='final_priority_score', ascending=False).head(20))

전체 유의 식별 판매자 수: 456명
                             seller_id product_category_name_english  seller_processing_days  Patience_Limit  final_priority_score  is_confirmed_risk
1834  7c67e1448b00f6e969d365cea6b010ab              office_furniture                   10.14             3.5            120.543615                  1
2629  7c67e1448b00f6e969d365cea6b010ab              office_furniture                   13.55             3.5            119.899348                  1
3735  a6fe7de3d16f6149ffe280349a8535a0       fashion_underwear_beach                    6.85             2.7            117.706416                  1
130   7c67e1448b00f6e969d365cea6b010ab              office_furniture                    7.40             3.5            115.597472                  1
1002  7c67e1448b00f6e969d365cea6b010ab              office_furniture                   13.58             3.5            115.456042                  1
2983  2cb4700db635baa1c0d4f90ed27b6669                          baby           